In [1]:
!pip install bayesian-optimization

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-py3-none-any.whl size=11685 sha256=5ada4b98643c9f07d3e07e8789e1d43833bc21b197fc934d0ba39be6078d8004
  Stored in directory: /root/.cache/pip/wheels/fd/9b/71/f127d694e02eb40bcf18c7ae9613b88a6be4470f57a8528c5b
Successfully built bayesian-optimization


In [5]:
class Data:
    def __init__(self, sensorPostions, space, epsilon):
        self.radius = 1
        self.placeHolders = sensorPostions
        self.epsilon = epsilon
        self.space = space
        # self.SensorPlaceHolderSetup()
        
    def frange(self, start, stop, step):
        steps = []
        while start <= stop:
            steps.append(start)
            start +=step
            
        return steps

    # def SensorPlaceHolderSetup(self):
    #     Xs = self.frange(0, self.space[0], self.epsilon)
    #     Ys = self.frange(0, self.space[1], self.epsilon)
            
    #     for x in Xs:
    #       for y in Ys:
    #         self.placeHolders.append([x, y])

    def GetSensorConfiguration(self):
        from collections import Counter
        sensorLocations, sensorTypes = self.GetSensorLocations()

        summaryDict = Counter(sensorTypes)

        # TODO: DIFFERENT SENSOR TYPE DEFINITIONS SHOULD BE ADDED HERE:
        configurationSummary = []
        for key in summaryDict:
            if (key == 1):
                configurationSummary.append(['motion sensors', summaryDict[key]])

            elif (key == 2):
                configurationSummary.append(['beacon sensors', summaryDict[key]])

        configurationDetails = []
        for index, loc in enumerate(sensorLocations):
            if (sensorTypes[index] == 1):
                configurationDetails.append(tuple([loc, 'kitchen', 'motion sensors']))

            elif (sensorTypes[index] == 2):
                configurationDetails.append(tuple([loc, 'kitchen', 'beacon sensors']))

        return [[configurationSummary, [tuple(configurationDetails)]], self.radius]


    def GetSensorLocations(self):
        sensorLocations = []
        sensorTypes = []
        for index, sensorIndicator in enumerate(self.placeHolders):
            sensorLocations.append(self.placeHolders[index])

            # TODO: DIFFERENT SENSOR TYPE DEFINITIONS SHOULD BE ADDED HERE:
            sensorTypes.append(1)


        return sensorLocations, sensorTypes

In [6]:
class BOVariables:
    def __init__(self, Data_path, epsilon, initSensorNum, maxSensorNum, radius, sampleSize):
        self.epsilon = epsilon
        self.Data_path = Data_path
        self.initSensorNum = initSensorNum
        self.maxSensorNum = maxSensorNum
        self.radius = radius
        self.sensor_distribution, self.types, self.space, self.rooms, self.agentTraces = self.ModelsInitializations()

    def ModelsInitializations(self):
        #----- Space and agent models -----: 
        simworldname = self.Data_path + '/Configuration Files/simulationWorld2.xml'
        agentTraces = []
        directory = os.fsencode(self.Data_path + 'Agent Trace Files/')
            
        for file in os.listdir(directory):
            filename = os.fsdecode(file)
            if filename.endswith(".csv"): 
                agentTraces.append(self.Data_path + 'Agent Trace Files/' + filename)

        # Parsing the space model: 
        space, rooms = pf.ParseWorld(simworldname)

        xs = []
        for i in space:
          for j in i:
            xs.append(j)
        A = list(set(xs))
        A.sort()
        space = [A[-1], A[-2]]

        # User parameters 
        types, sensor_distribution = pf.GetUsersParameters()

        roomsList = []
        for room in sensor_distribution:
            roomsList.append(room)
              
        return sensor_distribution, types, space, rooms, agentTraces


In [19]:
def frange(start, stop, step):
    steps = []
    while start <= stop:
        steps.append(start)
        start +=step
        
    return steps

def MakeSensorCombinations(start, end, epsilon, sensorType, room):
    a1, b1 = makeBoundaries(epsilon, start[0], end[0])
    a2, b2 = makeBoundaries(epsilon, start[1], end[1])    
    Xs = frange(a1, b1, epsilon)
    Ys = frange(a2, b2, epsilon)
    
    points = list(itertools.product(list(itertools.product(Xs, Ys)), [room], [sensorType[0]])) 
    C = itertools.combinations(points, distribution[room][types.index(sensorType)])

    return C

def PreProcessor(df):
    # df['motion sensors'] = df['motion sensors'].apply(ast.literal_eval)
    df['motion sensors'] = df['motion sensors'].apply(lambda s: list(map(int, s)))
    # df['beacon sensors'] = df['beacon sensors'].apply(ast.literal_eval)
    try:
      df['beacon sensors'] = df['beacon sensors'].apply(lambda s: list(map(int, s)))
    except:
      pass

    sensors = set([])

    previous_M = None
    previous_B = None
    output_file = []

    for index, row in df.iterrows():
      T = row['time']
      M = row['motion sensors']
      try:
        B = row['beacon sensors']
      except:
        pass

      Activity = row['activity']
      Activity = Activity.replace(' ', '_')
      MotionSensor_Names = []
      sensorNames = []
      MotionSensor_Message = []
      BeaconSensor_Names = []
      BeaconSensor_Message = []
      

      # time = convertTime(T)
      time = "2020-06-16 " + T + ".00"

      # Motion Sensor
      for i in range(len(M)):
        sensorNames.append(Name(i, 'M'))
        if M[i] == 1:
          if (previous_M != None):
            if (previous_M[i] == 0):
              MotionSensor_Names.append(Name(i,'M'))
              MotionSensor_Message.append('ON')

          else:
            MotionSensor_Names.append(Name(i,'M'))
            MotionSensor_Message.append('ON')

        if previous_M != None:
          if M[i] == 0 and previous_M[i] == 1:
            MotionSensor_Names.append(Name(i,'M'))
            MotionSensor_Message.append('OFF')

      previous_M = M
      # Beacon Sensor

      try:
        for i in range(len(B)):
          sensorNames.append(Name(i, 'B'))
          if B[i] == 1:
            BeaconSensor_Names.append(Name(i,'B'))
            BeaconSensor_Message.append('ON')
          if previous_B != None:
            if B[i] == 0 and previous_B[i] == 1: 
              BeaconSensor_Names.append(Name(i,'B'))
              BeaconSensor_Message.append('OFF')
        previous_B = B

      except:
        pass

      for m in range(len(MotionSensor_Names)):
        output_file.append(time +' '+ MotionSensor_Names[m] + ' ' + MotionSensor_Names[m] + ' ' + MotionSensor_Message[m] + ' ' + Activity)
        
      for s in sensorNames:
          sensors.add(s)

    return output_file, list(sensors)

#returns the name of the sensor
def Name(number, typeSensor):
    if number < 10:
      return typeSensor + str(0) + str(number)
    else:
      return typeSensor + str(number)

#converts epoch time to human readable
def convertTime(posix_timestamp):
    tz = pytz.timezone('MST')
    dt = datetime.fromtimestamp(posix_timestamp, tz)
    time = dt.strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]
    return time

def MakeDataBoundaries(height = 10.5, width = 6.6, MaxSensors = 15):
    from collections import defaultdict, OrderedDict

    d = dict()

    for idx in range(MaxSensors):
            d['x' + str(idx)] = (0.5, width - 0.5)
            d['y' + str(idx)] = (0.5, height - 0.5)

    return d

def black_box_function(sample, simulateMotionSensors = True, simulateEstimotes = False, Plotting = False):       
    files = []

    if (runningOnGoogleColab == True):
        sys.path.append('gdrive/My Drive/PhD/Thesis/Ideas/Codes/Sensor Simulator/')
        Data_path = 'gdrive/My Drive/PhD/Thesis/Ideas/Codes/Sensor Simulator/'
        
    else:
        sys.path.append('../../Codes/Sensor Simulator/')
        Data_path = '../../Codes/Sensor Simulator/'

    all_sensors = set([])

    for agentTrace in BOV.agentTraces:
        df_ = sim_sis.RunSimulator(BOV.space, BOV.rooms, agentTrace, sample.GetSensorConfiguration(), simulateMotionSensors, simulateEstimotes, Plotting, BOV.Data_path)
        dataFile, sensors = PreProcessor(df_)
        all_sensors.update(sensors)
        files.append(dataFile)
        
    
    if (runningOnGoogleColab == True):
        sys.path.append('gdrive/My Drive/PhD/Thesis/Ideas/Codes/CASAS/AL-Smarthome')

    else:
        sys.path.append('../../Codes/CASAS/AL-Smarthome')

    import al
    import imp
    imp.reload(al)
    all_sensors = list(all_sensors)

    accuracy = (al.leave_one_out(files, all_sensors)[0]) * 100

    if accuracy < 0:
        accuracy = 0

    return accuracy

def function_to_be_optimized(x0, y0, x1, y1, x2, y2, x3, y3, x4, y4, x5, y5, x6, y6, x7, y7, x8, y8, x9, y9, x10, y10, x11, y11, x12, y12, x13, y13, x14, y14):   
    allData = locals()

    sensorPositions = []
    sensor_xy = []
    for i, a in enumerate(allData):
        if (i % 2 == 0):
            sensor_xy.append(round(allData[a] * 2) / 2)
        else:
            sensor_xy.append(round(allData[a] * 2) / 2)
            sensorPositions.append(sensor_xy)
            sensor_xy = []
        
    data = Data(sensorPositions, BOV.space, epsilon)

    return black_box_function(data)

In [21]:
global runningOnGoogleColab
runningOnGoogleColab = True
    
if __name__ ==  '__main__':
    import sys

    if (runningOnGoogleColab == True):
        from google.colab import drive    
        drive.mount('/content/gdrive', force_remount=True)
        Data_path = 'gdrive/My Drive/PhD/Thesis/Ideas/Codes/Sensor Simulator/'
        sys.path.append('gdrive/My Drive/PhD/Thesis/Ideas/Codes/Sensor Simulator/')
        
    else:
        Data_path = '../../Codes/Sensor Simulator/'
        sys.path.append('../../Codes/Sensor Simulator/')

    from bayes_opt import BayesianOptimization
    from ipywidgets import IntProgress
    from IPython.display import display
    from scipy.stats import norm
    from numpy import argmax
    import SIM_SIS_Libraries.SensorsClass
    import SIM_SIS_Libraries.SIM_SIS_Simulator as sim_sis
    import SIM_SIS_Libraries.ParseFunctions as pf
    import itertools
    import numpy as np
    import pandas as pd
    import SIM_SIS_Libraries.PreDeploymentEvaluation as pde
    import copy
    from datetime import datetime
    import pytz
    import ast
    import os
    import random



    iteration = 1000
    epsilon = 1/2
    initSensorNum = 10
    maxSensorNum = 18
    radius = 1
    finalResults = []
    sampleSize = 100
    height = 10.5
    width = 6.6
    MaxSensors = 15

    kind="poi"
    kappa=10, 
    xi=0.0, 
    kappa_decay=0.995

    dataBoundaries = MakeDataBoundaries(height = height, width = width, MaxSensors = MaxSensors)

    f = IntProgress(min=0, max=iteration) # instantiate the bar
    display(f) # display the bar

    '''
    BOV =  BOVariables(
                    Data_path, 
                    epsilon, 
                    initSensorNum, 
                    maxSensorNum, 
                    radius,
                    sampleSize)

    optimizer = BayesianOptimization(
                    f=function_to_be_optimized,
                    pbounds=dataBoundaries,
                    verbose=2,
                    random_state=1)


    optimizer.maximize(
                    init_points = 10,
                    n_iter = iteration)

    print(optimizer.max)

    '''

    optimizer = BayesianOptimization(
                    f=None,
                    pbounds=dataBoundaries,
                    verbose=2,
                    random_state=1)
    

    # optimizer.maximize(n_iter=100)

    from bayes_opt import UtilityFunction

    # Hyperparameters are important here!
    utility = UtilityFunction(kind=kind, kappa=kappa, xi=xi, kappa_decay=kappa_decay)

    next_point_to_probe = optimizer.suggest(utility)
    # print("Next point to probe is:", next_point_to_probe)

    target = function_to_be_optimized(**next_point_to_probe)
    # print("Found the target value to be:", target)

    optimizer.register(
        params=next_point_to_probe,
        target=target,
    )


    accuracies = []
    iter = 0
    while iter < iteration:
        next_point = optimizer.suggest(utility)
        target = function_to_be_optimized(**next_point)
        optimizer.register(params=next_point, target=target)
        
        if target != 0:
            iter += 1
            accuracies.append(target)
            print(target, next_point)

    print(optimizer.max)

  
    

Mounted at /content/gdrive


IntProgress(value=0, max=1000)

47.71407539713753 {'x0': 4.079602827416506, 'x1': 4.25141305023371, 'x10': 1.3338267524505556, 'x11': 2.030368218611583, 'x12': 2.48773209185365, 'x13': 3.9143951680134554, 'x14': 4.3249074179323985, 'x2': 2.7673366225958578, 'x3': 1.142669908173347, 'x4': 2.9296128275522353, 'x5': 6.056887234425438, 'x6': 3.0774982437456346, 'x7': 5.926798643838053, 'x8': 4.326136387945054, 'x9': 4.248931820596097, 'y0': 8.242318530090389, 'y1': 8.561740116055597, 'y10': 1.066871737705383, 'y11': 2.216989658602832, 'y12': 8.011701057439574, 'y13': 7.574298299744261, 'y14': 3.2352736304514154, 'y2': 4.512777952190774, 'y3': 3.0385763541802837, 'y4': 6.240376536031465, 'y5': 7.320121078802176, 'y6': 9.456457566683719, 'y7': 2.62897186823354, 'y8': 9.540822839977977, 'y9': 3.3146871059515775}
42.41601843442784 {'x0': 5.572165461989462, 'x1': 4.345175923178278, 'x10': 2.9429118200845803, 'x11': 0.7259592558001075, 'x12': 1.0226695154705738, 'x13': 1.4428596780318226, 'x14': 1.32388942484911, 'x2': 3.85507

ValueError: ignored

In [ ]:
def PreProcessor(df):
        # df['motion sensors'] = df['motion sensors'].apply(ast.literal_eval)
        df['motion sensors'] = df['motion sensors'].apply(lambda s: list(map(int, s)))
        # df['beacon sensors'] = df['beacon sensors'].apply(ast.literal_eval)
        try:
          df['beacon sensors'] = df['beacon sensors'].apply(lambda s: list(map(int, s)))
        except:
          pass

        sensors = set([])

        previous_M = None
        previous_B = None
        output_file = []

        for index, row in df.iterrows():
          T = row['time']
          M = row['motion sensors']
          try:
            B = row['beacon sensors']
          except:
            pass

          Activity = row['activity']
          Activity = Activity.replace(' ', '_')
          MotionSensor_Names = []
          sensorNames = []
          MotionSensor_Message = []
          BeaconSensor_Names = []
          BeaconSensor_Message = []
          

          # time = convertTime(T)
          time = "2020-06-16 " + T + ".00"

          # Motion Sensor
          for i in range(len(M)):
            sensorNames.append(Name(i, 'M'))
            if M[i] == 1:
              if (previous_M != None):
                if (previous_M[i] == 0):
                  MotionSensor_Names.append(Name(i,'M'))
                  MotionSensor_Message.append('ON')

              else:
                MotionSensor_Names.append(Name(i,'M'))
                MotionSensor_Message.append('ON')

            if previous_M != None:
              if M[i] == 0 and previous_M[i] == 1:
                MotionSensor_Names.append(Name(i,'M'))
                MotionSensor_Message.append('OFF')

          previous_M = M
          # Beacon Sensor

          try:
            for i in range(len(B)):
              sensorNames.append(Name(i, 'B'))
              if B[i] == 1:
                BeaconSensor_Names.append(Name(i,'B'))
                BeaconSensor_Message.append('ON')
              if previous_B != None:
                if B[i] == 0 and previous_B[i] == 1: 
                  BeaconSensor_Names.append(Name(i,'B'))
                  BeaconSensor_Message.append('OFF')
            previous_B = B

          except:
            pass

          for m in range(len(MotionSensor_Names)):
            output_file.append(time +' '+ MotionSensor_Names[m] + ' ' + MotionSensor_Names[m] + ' ' + MotionSensor_Message[m] + ' ' + Activity)
            
          for s in sensorNames:
              sensors.add(s)

        return output_file, list(sensors)

#returns the name of the sensor
def Name(number, typeSensor):
    if number < 10:
      return typeSensor + str(0) + str(number)
    else:
      return typeSensor + str(number)

#converts epoch time to human readable
def convertTime(posix_timestamp):
    tz = pytz.timezone('MST')
    dt = datetime.fromtimestamp(posix_timestamp, tz)
    time = dt.strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]
    return time

def ModelsInitializations(Data_path):
    #----- Space and agent models -----: 
    simworldname = Data_path + '/Configuration Files/simulationWorld2.xml'
    agentTraces = []
    directory = os.fsencode(Data_path)

    print(directory)
    
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        if filename.endswith(".csv"): 
            agentTraces.append(Data_path + filename)

    # Parsing the space model: 
    space, rooms = pf.ParseWorld(simworldname)

    xs = []
    for i in space:
      for j in i:
        xs.append(j)
    A = list(set(xs))
    A.sort()
    space = [A[-1], A[-2]]

    # User parameters 
    types, sensor_distribution = pf.GetUsersParameters()

    roomsList = []
    for room in sensor_distribution:
        roomsList.append(room)

    return sensor_distribution, types, space, rooms, agentTraces


def Train(Data_path, bestChromosome):
    sensor_distribution, types, space, rooms, agentTraces = ModelsInitializations(Data_path + 'Agent Trace Files/')
    files = []


    if (runningOnGoogleColab ==True):
        sys.path.append('gdrive/My Drive/PhD/Thesis/Ideas/Codes/Sensor Simulator/')
        Data_path = 'gdrive/My Drive/PhD/Thesis/Ideas/Codes/Sensor Simulator/'
    else:
        sys.path.append('../../Codes/Sensor Simulator/')
        Data_path = '../../Codes/Sensor Simulator/'

    all_sensors = set([])

    for agentTrace in agentTraces:
        filecoding = ' '#"_" + str(iteration) + "_c" + str(index + 1) + '(' + self.mode + ')'
        df_ = sim_sis.RunSimulator(space, rooms, agentTrace, bestChromosome.GetSensorConfiguration() , simulateMotionSensors, simulateEstimotes, Plotting, filecoding , Data_path)
        dataFile, sensors = PreProcessor(df_)
        all_sensors.update(sensors)
        #self.D = dataFile
        files.append(dataFile)


    if (runningOnGoogleColab == True):
        sys.path.append('gdrive/My Drive/PhD/Thesis/Ideas/Codes/CASAS/AL-Smarthome')
        # %cd 'gdrive/My Drive/PhD/Thesis/Ideas/Codes/CASAS/AL-Smarthome'

    else:
        sys.path.append('../../Codes/CASAS/AL-Smarthome')

    #import al
    #import imp
    imp.reload(al)
    all_sensors = list(all_sensors)

    cf = al.Train(files, all_sensors)
    
    return cf
    
    
    # chromosome.fitness = (al.leave_one_out(files, all_sensors)[0] - (sum(chromosome.grid) / 100)) * 100 # - (len(chromosome.placeHolders)**(np.sum(chromosome.grid)/len(chromosome.placeHolders))) / len(chromosome.placeHolders)) * 100
    # if chromosome.fitness < 0:
    #     chromosome.fitness = 0


def Test(cf, bestChromosome, Data_path):
    sensor_distribution, types, space, rooms, agentTraces = ModelsInitializations(Data_path + 'Agent Trace Files Test/')
    files = []

    if (runningOnGoogleColab == True):
        sys.path.append('gdrive/My Drive/PhD/Thesis/Ideas/Codes/Sensor Simulator/')
        Data_path = 'gdrive/My Drive/PhD/Thesis/Ideas/Codes/Sensor Simulator/'
    
    else:
        sys.path.append('../../Codes/Sensor Simulator/')
        Data_path = '../../Codes/Sensor Simulator/'

    all_sensors = set([])

    for agentTrace in agentTraces:
        filecoding = ' '#"_" + str(iteration) + "_c" + str(index + 1) + '(' + self.mode + ')'
        df_ = sim_sis.RunSimulator(space, rooms, agentTrace, bestChromosome.GetSensorConfiguration() , simulateMotionSensors, simulateEstimotes, Plotting, filecoding , Data_path)
        dataFile, sensors = PreProcessor(df_)
        all_sensors.update(sensors)
        #self.D = dataFile
        files.append(dataFile)

    if (runningOnGoogleColab == True):
        sys.path.append('gdrive/My Drive/PhD/Thesis/Ideas/Codes/CASAS/AL-Smarthome')
        # %cd 'gdrive/My Drive/PhD/Thesis/Ideas/Codes/CASAS/AL-Smarthome'
    
    else:
        sys.path.append('../../Codes/CASAS/AL-Smarthome')

    #import al
    #import imp
    imp.reload(al)
    all_sensors = list(all_sensors)

    result = al.Test(cf, files, all_sensors)
    
    print(result)
    
    
    
    # chromosome.fitness = (al.leave_one_out(files, all_sensors)[0] - (sum(chromosome.grid) / 100)) * 100 # - (len(chromosome.placeHolders)**(np.sum(chromosome.grid)/len(chromosome.placeHolders))) / len(chromosome.placeHolders)) * 100
    # if chromosome.fitness < 0:
    #     chromosome.fitness = 0


allData = optimizer.max['params']
sensorPositions = []

sp = []
for i, a in enumerate(allData):
    
    if (i % 2 == 0):
        sp.append(allData[a])
    else:
        sp.append(allData[a])
        sensorPositions.append(sp)
        sp = []
    
data = Data(sensorPositions, BOV.space, epsilon)




bestdata = Data(sensorPositions, BOV.space, epsilon)
# black_box_function(data)



import al
import imp
simulateMotionSensors, simulateEstimotes, Plotting, iteration = True, False, False, 1
bestChromosome = bestdata

if (runningOnGoogleColab == True):
    Data_path = 'gdrive/My Drive/PhD/Thesis/Ideas/Codes/Sensor Simulator/'

else:
    Data_path = '../../Codes/Sensor Simulator/'

cf = Train(Data_path, bestChromosome)
print('Training Done!')
Test(cf, bestChromosome, Data_path)


In [ ]:
'''
def generateListNames(MaxSensors = 15):
    s = ''
    for i in range(MaxSensors):
      s += 'x' + str(i) + ', ' + 'y' + str(i) + ', '
    
    s = s[:-2]
    print(s)

generateListNames()
'''

In [ ]:
x = []
y = []

for key in allData:
    if 'x' in key:
        x.append(allData[key])

    else:
        y.append(allData[key])

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(8, 10))
plt.scatter(x, y)